In [ ]:
from pprint import pprint
import pickle
from scipy.stats import skew
import seaborn as sns
import PI_class_EnbPI_journal as EnbPI
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from pyod.models.pca import PCA
from pyod.models.ocsvm import OCSVM
from pyod.models.iforest import IForest
from pyod.models.hbos import HBOS
from pyod.models.knn import KNN  # kNN detector
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import neighbors
from sklearn.neural_network import MLPClassifier
from sklearn import svm
import utils_EnbPI_journal as util
from matplotlib.lines import Line2D  # For legend handles
import calendar
import warnings
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import itertools
import importlib
import time
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm


In [ ]:
warnings.filterwarnings("ignore")
importlib.reload(sys.modules["PI_class_EnbPI_journal"])


In [ ]:


"""Sec 5.2 Figure 3 on comparing with time-series methods
   Silimar results in the appendix are included as well"""


def big_transform(CA_cities, current_city, one_dim, d, train_size):
    # Used for California data
    # Next, merge these data (so concatenate X_t and Y_t for one_d or not)
    # Return [X_train, X_test, Y_train, Y_test] from data_x and data_y
    # Data_x is either multivariate (direct concatenation)
    # or univariate (transform each series and THEN concatenate the transformed series)
    big_X_train = []
    big_X_predict = []
    for city in CA_cities:
        data = eval(f"data{city}")  # Pandas DataFrame
        data_x = data.loc[:, data.columns != "DHI"]
        data_y = data["DHI"]
        data_x_numpy = data_x.to_numpy()  # Convert to numpy
        data_y_numpy = data_y.to_numpy()  # Convert to numpy
        X_train = data_x_numpy[:train_size, :]
        X_predict = data_x_numpy[train_size:, :]
        Y_train_del = data_y_numpy[:train_size]
        Y_predict_del = data_y_numpy[train_size:]
        if city == current_city:
            Y_train = Y_train_del
            Y_predict = Y_predict_del
        if one_dim:
            X_train, X_predict, Y_train_del, Y_predict_del = util.one_dimen_transform(
                Y_train_del, Y_predict_del, d=d
            )
            big_X_train.append(X_train)
            big_X_predict.append(X_predict)
            if city == current_city:
                Y_train = Y_train_del
        else:
            big_X_train.append(X_train)
            big_X_predict.append(X_predict)
    X_train = np.hstack(big_X_train)
    X_predict = np.hstack(big_X_predict)
    return [X_train, X_predict, Y_train, Y_predict]


# Read data and initialize parameters
result_type = "Fig3"
response_ls = {
    "Solar_Atl": "DHI",
    "Palo_Alto": "DHI",
    "Wind_Austin": "MWH",
    "green_house": 15,
    "appliances": "Appliances",
    "Beijing_air": "PM2.5",
}
if result_type == "Fig3":
    # Figure 3
    max_data_size = 10000
    dataSolar_Atl = util.read_data(3, "Data/Solar_Atl_data.csv", max_data_size)
    Data_name = ["Solar_Atl"]
    CA_energy_data = False
elif result_type == "AppendixB3":
    # Results in Appendix B.3
    CA_cities = [
        "Fremont",
        "Milpitas",
        "Mountain_View",
        "North_San_Jose",
        "Palo_Alto",
        "Redwood_City",
        "San_Mateo",
        "Santa_Clara",
        "Sunnyvale",
    ]
    for city in CA_cities:
        globals()["data%s" % city] = util.read_CA_data(f"Data/{city}_data.csv")
    Data_name = ["Palo_Alto"]
    CA_energy_data = True
else:
    # Results in Appendix B.4
    datagreen_house = util.read_data(0, "Data/green_house_data.csv", max_data_size)
    dataappliances_data = util.read_data(1, "Data/appliances_data.csv", max_data_size)
    dataBeijing_air = util.read_data(
        2, "Data/Beijing_air_Tiantan_data.csv", max_data_size
    )
    Data_name = ["green_house", "appliances", "Beijing_air"]



In [ ]:
min_alpha = 0.0001
max_alpha = 10
ridge_cv = RidgeCV(alphas=np.linspace(min_alpha, max_alpha, 10))
lasso_cv = LassoCV(alphas=np.linspace(min_alpha, max_alpha, 10))
random_forest = RandomForestRegressor(
    n_estimators=10, criterion="squared_error", bootstrap=False, max_depth=2, n_jobs=-1
)
extra_trees = ExtraTreesRegressor(n_estimators=10, criterion="squared_error", bootstrap=False, max_depth=2, n_jobs=-1)

In [ ]:
dataSolar_Atl

In [ ]:
pprint(list(dataSolar_Atl))

In [ ]:
#alpha_ls = [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.50, 0.75, 0.80, 0.85, 0.90, 0.95, 0.99]#np.linspace(0.05, 0.50, 10)
alpha_ls = [0.01, 0.10, 0.20, 0.50, 0.80, 0.90, 0.99]#np.linspace(0.05, 0.50, 10)
stride = 1


In [ ]:
importlib.reload(sys.modules['PI_class_EnbPI_journal'])


In [ ]:
'''
# First run time-series methods
for data_name in Data_name:
    one_dim = True
    # `d` is the num_lookbacks for AR-transformer
    d = 24
    itrial = 0
    data = eval(f'data{data_name}')  # Pandas DataFrame
    data_x = data.loc[:, data.columns != response_ls[data_name]]
    data_y = data[response_ls[data_name]]
    total_data_points = data_x.shape[0]
    train_size = int(0.2 * total_data_points)
    results_ts = pd.DataFrame(columns=['itrial', 'dataname',
                                       'method', 'alpha', 'coverage', 'width'])
    np.random.seed(98765 + itrial)
    for alpha in alpha_ls:
        print(f'At trial # {itrial} and alpha={alpha}')
        print(f'For {data_name}')
        if CA_energy_data:
            X_train, X_predict, Y_train, Y_predict = big_transform(
                Data_name, data_name, one_dim, d, train_size)
        else:
            # for one-step ahead forecasting. 
            #TODO: generalize this for multistep forecasting
            data_y = data_y.shift(-1)
            data_y.dropna(inplace=True)
            data_x.drop(data_x.tail(1).index, inplace=True)

            data_x_numpy = data_x.to_numpy()  # Convert to numpy
            data_y_numpy = data_y.to_numpy()  # Convert to numpy
            X_train = data_x_numpy[:train_size, :]
            X_predict = data_x_numpy[train_size:, :]
            Y_train = data_y_numpy[:train_size]
            Y_predict = data_y_numpy[train_size:]
        
        ridge_results = EnbPI.prediction_interval(
            fit_func=ridge_cv,  X_train=X_train, X_predict=X_predict, Y_train=Y_train, Y_predict=Y_predict, bootstrap_type="circular", block_length=24)
        # For ARIMA and other time-series methods, only run once
        result_ts = ridge_results.run_experiments(
            alpha, stride, data_name, itrial, none_CP=True)
        result_ts.rename(columns={'train_size': 'alpha'}, inplace=True)
        if CA_energy_data:
            result_ts['alpha'].replace(
                train_size - d, alpha, inplace=True)
        else:
            result_ts['alpha'].replace(
                train_size, alpha, inplace=True)
        results_ts = pd.concat([results_ts, result_ts])
        results_ts.to_csv(
            f'Results/{data_name}_many_alpha_new_tseries.csv', index=False)
'''

In [ ]:
#results_ts

In [ ]:
# Then run conformal-related methods
stride = 1
# `d` is num_lookbacks for AR-transformer
d = 24
miss_test_idx = []
#alpha = 0.1
tot_trial = 1  # For CP methods that randomizes (for EnbPI, should be set to 1)
B = 25  # number of bootstrap samples
#rnn = True
methods = ['Ensemble']#, 'ICP', 'Weighted_ICP']
bootstrap_types = ["circular", "stationary", "moving", "random"]#, "random"]
block_lengths = [6, 12, 24, 48, 60, 72]
# NOTE, if want to run J+aB (Kim et al. 2020), then let
# methods = ['Ensemble', 'ICP', 'Weighted_ICP', 'JaB']

#the one_dim thing just means whether to use the
for one_dim in [True, False]:
    for data_name in Data_name:
        data = eval(f'data{data_name}')  # Pandas DataFrame
        data_x = data.loc[:, data.columns != response_ls[data_name]]
        data_y = data[response_ls[data_name]]
        total_data_points = data_x.shape[0]
        #train_size = int(0.10 * total_data_points)
        Train_size = np.linspace(0.1 * total_data_points,
                                 0.2 * total_data_points, 2).astype(int)
        #Train_size = [Train_size[0], Train_size[4], Train_size[8]]
        results = pd.DataFrame(columns=['itrial', 'dataname', 'muh_fun',
                                        'method', "train_size", 'alpha', 'coverage', 'width', "bootstrap_type", "block_length"])
        for itrial in range(tot_trial):
            np.random.seed(98765 + itrial)
            for train_size in Train_size:
                # Note, this is necessary because a model may "remember the past"
                #nnet = util.keras_mod()
                #rnnet = util.keras_rnn()
                print(f'For {data_name}')
                print(f'At trial # {itrial} and train_size={train_size}')
                if CA_energy_data:
                    X_train, X_predict, Y_train, Y_predict = big_transform(
                        Data_name, data_name, one_dim, d, train_size)
                else:
                    # for one-step ahead forecasting. 
                    #TODO: generalize this for multistep forecasting
                    data_y = data_y.shift(-1)
                    data_y.dropna(inplace=True)
                    data_x.drop(data_x.tail(1).index, inplace=True)

                    data_x_numpy = data_x.to_numpy()  # Convert to numpy
                    data_y_numpy = data_y.to_numpy()  # Convert to numpy

                    X_train = data_x_numpy[:train_size, :]
                    X_predict = data_x_numpy[train_size:, :]
                    Y_train = data_y_numpy[:train_size]
                    Y_predict = data_y_numpy[train_size:]
                    if one_dim:
                        X_train, X_predict, Y_train, Y_predict = util.one_dimen_transform(
                            Y_train, Y_predict, d=d)

                for bootstrap_type in tqdm(bootstrap_types):
                    for block_length in tqdm(block_lengths):
                        lasso_results = EnbPI.prediction_interval(fit_func=lasso_cv,  X_train=X_train, X_predict=X_predict, Y_train=Y_train, Y_predict=Y_predict, bootstrap_type=bootstrap_type, block_length=block_length, random_seed=B+itrial)
                        lasso_results.fit_bootstrap_models_online(B, miss_test_idx)
                        
                        ridge_results = EnbPI.prediction_interval(
                            fit_func=ridge_cv,  X_train=X_train, X_predict=X_predict, Y_train=Y_train, Y_predict=Y_predict, bootstrap_type=bootstrap_type, block_length=block_length, random_seed=B+itrial)
                        ridge_results.fit_bootstrap_models_online(B, miss_test_idx)
                        
                        rf_results = EnbPI.prediction_interval(
                            fit_func=random_forest,  X_train=X_train, X_predict=X_predict, Y_train=Y_train, Y_predict=Y_predict, bootstrap_type=bootstrap_type, block_length=block_length, random_seed=B+itrial)
                        rf_results.fit_bootstrap_models_online(B, miss_test_idx)

                        et_results = EnbPI.prediction_interval(
                            fit_func=extra_trees,  X_train=X_train, X_predict=X_predict, Y_train=Y_train, Y_predict=Y_predict, bootstrap_type=bootstrap_type, block_length=block_length, random_seed=B+itrial)
                        et_results.fit_bootstrap_models_online(B, miss_test_idx)

                        for alpha in alpha_ls:
                            # Note, this is necessary because a model may "remember the past"
                            print(f'At trial # {itrial} and alpha={alpha}')
                            print(f'For {data_name}')
                            
                            # CP Methods
                            print(f'regressor is {lasso_cv.__class__.__name__}')
                            result_lasso = lasso_results.run_experiments(
                                alpha, stride, data_name, itrial, methods=methods)
                            
                            print(f'regressor is {extra_trees.__class__.__name__}')
                            result_et = et_results.run_experiments(
                                alpha, stride, data_name, itrial, methods=methods)
                            
                            print(f'regressor is {ridge_cv.__class__.__name__}')
                            result_ridge = ridge_results.run_experiments(
                                alpha, stride, data_name, itrial, methods=methods)
                            
                            print(f'regressor is {random_forest.__class__.__name__}')
                            result_rf = rf_results.run_experiments(
                                alpha, stride, data_name, itrial, methods=methods)
                            
                            results_now = pd.concat(
                                    [result_lasso, result_ridge, result_rf, result_et])
                            results_now.rename(
                                columns={'train_size': 'alpha'}, inplace=True)
                            
                            if one_dim:
                                results_now['alpha'].replace(
                                    train_size - d, alpha, inplace=True)
                            else:
                                results_now['alpha'].replace(
                                    train_size, alpha, inplace=True)
                            
                            results_now["block_length"] = block_length
                            results_now["bootstrap_type"] = bootstrap_type
                            results = pd.concat([results, results_now])

        if one_dim:
            results.to_csv(
                f'Results/{data_name}_many_alpha_many_train_new_1d.csv', index=False)
        else:
            results.to_csv(
                f'Results/{data_name}_many_alpha_many_train_new.csv', index=False)


In [ ]:
#results[(results["muh_fun"]=="RidgeCV") & (results["alpha"]==0.05)]

In [ ]:
#results[(results["muh_fun"]=="RidgeCV") & (results["alpha"]==0.05)]

In [ ]:
#results_ar = pd.read_csv(f'Results/{data_name}_many_alpha_new_1d.csv')

In [ ]:
#results_ar[(results_ar["muh_fun"]=="RandomForestRegressor") & (results_ar["alpha"]==0.25)]

In [ ]:
##X_train.shape

In [ ]:
#boot_samples_idx = util.generate_bootstrap_samples(
            len(X_train), len(X_train), B, bootstrap_type="nonoverlapping", block_length=6, random_seed=1)
#boot_samples_idx

In [ ]:
data_x

In [ ]:
#data_x.drop(data_x.tail(1).index)

In [ ]:
#one_dim